In [1]:
import numpy as np
import camb
import matplotlib.pyplot as plt

In [45]:
def get_eq(om0h2, theta):
    z_eq = 2.5*1e4 * om0h2 / theta**4
    k_eq = 7.46*1e-2 * om0h2 / theta**2  # in Mpc^-1
    return z_eq, k_eq
def get_zdrag(om0h2, ombh2):
    b1 = 0.313*(om0h2)**-0.419*(1+0.607*om0h2**0.674)
    b2 = 0.238*(om0h2)**0.223
    zdrag = 1291*om0h2**0.251/(1+0.659*om0h2**0.828)*(1+b1*ombh2**b2)
    return zdrag
def get_s(om0h2, ombh2, theta):
    """
    calculates the sound horizon at drag epoch in comoving distance


    """
    zdrag = get_zdrag(om0h2, ombh2)
    z_eq, k_eq = get_eq(om0h2, theta)
    Req = 31.5*ombh2*(theta)**-4 * (1e3/z_eq)
    Rdrag = 31.5*ombh2*(theta)**-4 * (1e3/zdrag)
    s = (
    (2./(3*k_eq))*np.sqrt(6./Req)*
    np.log((np.sqrt(1+Rdrag) + np.sqrt(Rdrag + Req))/(1+np.sqrt(Req)))
    )
    return s

In [47]:
theta = 2.728/2.7
keq = get_eq(0.2, theta)[1]
s = get_s(0.2, 0.04, theta)
print(keq*s/np.pi)

0.5875995068283334


In [19]:
class EisensteinHu:
    def __init__(self, ombh2=0.022, omch2=0.122,h=0.675, mnu=0.06, ns=0.965, As=2e-9, tau=0.06, TCMB=2.728):
        self.name = "Eisenstein-Hu"
        self.params = {
            'ombh2': ombh2,
            'omch2': omch2,
            'h': h,
            'mnu': mnu,
            'ns': ns,
            'As': As,
            'tau': tau,
            'TCMB': TCMB
        }
        self.om0h2 = ombh2 + omch2
        self.theta = TCMB/2.7
        self.zeq = 2.5*1e4 * self.om0h2 * (self.theta)**-4
        self.keq = 7.46*1e-2 * self.om0h2*(self.theta)**-2 # in Mpc^-1

        # k_silk calculation
        self.ksilk = 1.6*ombh2**0.52*self.om0h2**0.73*(1+(10.4*self.om0h2)**-0.95) # in Mpc^-1

    # zdrag calculation
    def get_zdrag(self):
        self.ombh2 = self.params['ombh2']
        b1 = 0.313*(self.om0h2)**-0.419*(1+0.607*self.om0h2**0.674)
        b2 = 0.238*(self.om0h2)**0.223
        self.zdrag = 1291*self.om0h2**0.251/(1+0.659*self.om0h2**0.828)*(1+b1*self.ombh2**b2)
        return self.zdrag
    
    # s calculation    
    def get_s(self):
        """
        calculates the sound horizon at drag epoch in comoving distance


        """
        self.zdrag = self.get_zdrag() # includes self.zdrag, self.ombh2
        self.Req = 31.5*self.ombh2*(self.theta)**-4 * (1e3/self.zeq)
        self.Rdrag = 31.5*self.ombh2*(self.theta)**-4 * (1e3/self.zdrag)
        self.s = (
        (2./(3*self.keq))*np.sqrt(6./self.Req)*
        np.log((np.sqrt(1+self.Rdrag) + np.sqrt(self.Rdrag + self.Req))/(1+np.sqrt(self.Req)))
        )
        return self.s
    # 
    def Transfer_function(self, k):
        self.omch2 = self.params['omch2']
        # calculate q and s
        
        self.get_s() # updates self.zdrag, self.ombh2, self.s

        # calculate nuisance parameters for alpha_c
        a1 = (46.9*self.ombh2)**0.670*(1+(32.1*self.om0h2)**-0.532)
        a2 = (12.0*self.ombh2)**0.424*(1+(45.0*self.om0h2)**-0.582)
        # calculate alpha_c
        alpha_c = a1**-(self.ombh2/self.om0h2) * a2**-(self.ombh2/self.om0h2)**3
        print("alpha_c:", alpha_c)

        # calculate nuisance parameters for beta_c
        b1 = 0.944*(1+(458*self.om0h2)**-0.708)**-1
        b2 = (0.395*self.om0h2)**-0.0266
        # calculate beta_c
        beta_c = 1/(1+b1*((self.omch2/self.om0h2)**b2 - 1))
        print("beta_c:", beta_c)

        # calculate T_c
        def Ttilde0(k, alpha_c, beta_c):
            q = k/(13.41*self.keq)
            C = 14.2*alpha_c+386./(1+69.9*q**1.08)
            return np.log(np.e+1.8*beta_c*q)/(np.log(np.e+1.8*beta_c*q)+C*q**2)
        f = 1/(1+(k*self.s/5.4)**4)

        T_c = f*Ttilde0(k, 1, beta_c) + (1-f)*Ttilde0(k, alpha_c, beta_c)


        # calculate alpha_b
        def G(y):
            return y*(-6.*np.sqrt(1+y)+(2+3*y)*np.log((np.sqrt(1+y)+1)/np.sqrt(1+y)-1))
        alpha_b = 2.07*self.keq*self.s*(1+self.Rdrag)**-(3/4)*G((1+self.zeq)/(1+self.zdrag))
        def j0(x):
            return np.sin(x)/x
        # calculate T_b
        beta_node = 8.41*self.om0h2**0.435
        stilde = self.s/(1+(beta_node/k/self.s)**3)**(1/3)
        beta_b = 0.5 + self.ombh2/self.om0h2 + (3 - 2*self.ombh2/self.om0h2)*np.sqrt((17.2*self.om0h2)**2 + 1)

        T_b = ((Ttilde0(k,1,1)/(1+(k*self.s/5.2)**2)) + (alpha_b/(1+(beta_b/k/self.s)**3)*np.exp(-(k/self.ksilk)**1.4)))*j0(k*stilde)

        T_k = self.ombh2/self.om0h2*T_b + self.omch2/self.om0h2*T_c
        return T_k
    
    def Power_spectrum(self, k, kpivot=0.05):
        self.ns = self.params['ns']
        self.As = self.params['As']
        T_k = self.Transfer_function(k)
        P_k = self.As*(k/kpivot)**(self.ns-1)*T_k**2
        return P_k
    

In [35]:
om0h2arr = np.linspace(0., 0.5, 10)
fracarr = np.linspace(0., 1., 10)
ombh2arr = om0h2arr[:, None]*fracarr[None, :]
om0h2arr = om0h2arr[:, None]*np.ones_like(fracarr)[None, :]

In [ ]:
4